In [47]:
import csv
import sys

import pandas as pd

In [36]:
sys.path.append('..')

In [37]:
from stigmapyze.common.reddit import Comment, Submission

In [33]:
STIGMA_HEADER = [
    'ID', 'Stig_c1', 'Stig_c2', 'Stig_c3', 'Stig_c4', 'Stig_c5',
    'Challn_c1', 'Challn_c2', 'Challn_c3', 'Challn_c4', 'Challn_c5']

In [3]:
stg = pd.read_csv('../data/input/2021-03-02T19:23:10/2021-03-02T19:23:10-stigma.csv')
cmt = pd.read_csv('../data/input/2021-03-02T19:23:10/2021-03-02T19:23:10-comments.csv')
sub = pd.read_csv('../data/input/2021-03-02T19:23:10/2021-03-02T19:23:10-submissions.csv')

In [5]:
cmt.link_id = cmt.link_id.str.extract('t3_(.*)')

In [58]:
subcmt = sub[['id']].merge(cmt, left_on='id', right_on='link_id', suffixes=['_sub', '_cmt'])
subcmt = subcmt.drop('id_sub', axis=1).rename({'id_cmt': 'id'}, axis=1)
# subcmt.link_id = subcmt.link_id.apply(lambda t: f't3_{t}')
subcmt = subcmt[subcmt.is_submitter == False]
subcmt.head()

,created_utc,id,score,author,is_submitter,parent_id,link_id,body
0,2021-03-02T19:31:43,gph9dfy,1,the-big-peper,False,lwgnzw,lwgnzw,Please do not do what ever you are about to do...
1,2021-03-03T00:23:21,gpi5kze,1,PotentialAd375,False,lwgnzw,lwgnzw,me and big pepper are here to talk to you
2,2021-03-02T19:28:51,gph919t,1,Xkeletal,False,lwgo4f,lwgo4f,My life went downhill during my a levels. What...
3,2021-03-02T19:37:05,gph9zt7,1,YouChemical4215,False,lwgo4f,lwgo4f,if there is a downhill in life there always mu...
5,2021-03-02T19:50:13,gphbj8i,2,Xkeletal,False,gphajg9,lwgo4f,I see. The good thing here is you're honest ab...


In [56]:
subcmt.to_csv(f'{csvbase}/new_comments.csv')

In [59]:
newsub = sub.drop('comments', axis=1).merge(pd.DataFrame(subcmt.groupby('link_id').apply(lambda g: list(g.id))), left_on='id', right_index=True).rename({0: 'comments'}, axis=1)
newsub.head()

,created_utc,id,score,author,title,full_link,selftext,comments
0,2021-03-02T19:25:23,lwgnzw,1,Gocrazygostupid18,How Do I Hang Myself With A Belt In A Walk-In ...,https://www.reddit.com/r/SuicideWatch/comments...,"I really want to die, I feel like there’s noth...","[gph9dfy, gpi5kze]"
1,2021-03-02T19:25:34,lwgo4f,1,violet-wakes-to-dawn,I'm a fucking failure,https://www.reddit.com/r/SuicideWatch/comments...,I'm in year 11 but my entire high school atten...,"[gph919t, gph9zt7, gphbj8i]"
2,2021-03-02T19:26:11,lwgoi7,1,Ok_Double_7030,How can I make my death easier on the people I...,https://www.reddit.com/r/SuicideWatch/comments...,I have decided that I no longer care about liv...,"[gph998g, gphbrdp, gphcf6h, gphcolr, gphcxx3, ..."
3,2021-03-02T19:26:15,lwgoja,1,Xkeletal,"It is my birthday. I am steadfast, in the vein...",https://www.reddit.com/r/SuicideWatch/comments...,I am a highschool dropout and college dropout....,"[gpha6hd, gphbnq1]"
5,2021-03-02T19:34:39,lwgu79,1,ISimplyHateUsernames,So many people say don't do it because you fam...,https://www.reddit.com/r/SuicideWatch/comments...,From young age until basically now I had alway...,[gpmk7aa]


In [60]:
newsub.to_csv(f'{csvbase}/new_submissions.csv')

In [45]:
csvbase = '../data/input/2021-03-02T19:23:10/'

In [49]:
def stigma_row(id: str, t) -> dict:
    row = {col: '' for col in STIGMA_HEADER}
    row['ID'] = f'{t} {id}'
    return row


94

In [51]:
newsub.to_csv(f'{csvbase}/new_submissions.csv')
subcmt.to_csv(f'{csvbase}/new_comments.csv')

In [61]:
stgf = open(f'{csvbase}/new_stigma.csv', 'w')
stg_csv = csv.DictWriter(stgf, fieldnames=STIGMA_HEADER)
stg_csv.writeheader()

for row in newsub.itertuples():
    stg_csv.writerow(stigma_row(row.id, 'Submission'))
    for cid in row.comments:
        stg_csv.writerow(stigma_row(cid, 'Comment'))

stgf.close()